# Site percolation using tree-based union/find algorithm and finite size scaling

## Library and functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def num_of_zero_in_list(lists):
    '''
    Count number of 0 in a list.
    '''
    x = 0
    for component in lists:
        if component == 0:
            x += 1
    return x

def num_in_dict(dictionary, number):
    '''
    Count number of components in a dictionary which is included key value of "number".
    '''
    a_list = dictionary[number]
    L = len(a_list)
    return L

def dict_to_matrix(lattice_size, dictionary):
    '''
    Convert label_dictionary into matrix.
    '''
    matrix = np.zeros((lattice_size, lattice_size))
    for label in dictionary:
        if label != 0:
            lits = dictionary[label]
            for coordi in lits:
                x, y = coordi[0], coordi[1] 
                matrix[x,y] = label
        else:
            continue
    matrix = matrix
    return matrix

def near_by_sites(lists):
    '''
    Find occupied neighbors.
    '''
    x = []
    for component in lists:
        if component != 0:
            x.append(component)
    return x

def multi(lists):
    '''
    Calculate multiplicities in a list.
    '''
    mul = []
    for component in lists:
        if lists.count(component) == 1:
            mul.append(1)
        elif lists.count(component) == 2:
            mul.append(2)
        elif lists.count(component) == 3:
            mul.append(3)
        elif lists.count(component) == 4:
            mul.append(4)
    return mul

def add_a_site(configuration):
    '''
    Add a randomly chosen site in a square lattice.
    '''
    L = len(configuration)
    lin_con = configuration.reshape(L**2)
    while True:
        c = np.random.randint(0,L**2)
        if lin_con[c] == 0:
            lin_con[c]= 2
            break
    configuration = lin_con.reshape((L,L))
    return configuration

def number_of_cluster(dictionary):
    '''
    Compute number of cluster in a given configuration.
    '''
    number_of_clust = 0
    for key in dictionary:
        number_of_clust += 1
    return number_of_clust

def giant_cluster_size(dictionary):
    '''
    Calculate the size of giant cluster in a given configuration.
    '''
    size_of_clusters = []
    for key in dictionary:
        size_of_clusters.append(len(dictionary[key]))
    m = max(size_of_clusters)
    return m

def cluster_sizes(dictionary):
    '''
    To calculate susceptibility in percolation, we should compute mean cluster size of 
    the configuration except for the largest cluster. 
    '''
    number_of_clusters = len(dictionary)
    what_sizes = []
    for key in dictionary:
        what_sizes.append(len(dictionary[key])) # a list contains size of each cluster
        
    maximum_cluster_size = max(what_sizes) #the largest cluster must be omitted from the summation
    what_sizes.remove(maximum_cluster_size)
    what_sizes = (np.array(what_sizes))**2
    Size = np.sum(what_sizes)
    
    return Size

### Union/find algorithm

In [3]:
def union_find(lattice_size, number_of_occupied_sites):
    '''
    Tree-based union/find algorithm which is more efficient than conventional algorithms.
    This code also contain calculation of giant cluster size and average cluster size of the input configuration.
    Some part doesn't include optimization for faster code run. If you need to make a configuration faster, use # code.
    '''
    L = lattice_size
    configuration, label_matrix = np.zeros((L,L)), np.zeros((L,L))
    label = 0
    label_dict = {} #an empty dictionary
    gc_ratio = []
    mean_cluster = []
    
    
    for n in range(number_of_occupied_sites):
        if n == 0:
            gc_ratio.append(0)
            mean_cluster.append(0)

        elif n > 0:
            add_a_site(configuration)

            for i in range(L):
                for j in range(L):
                    if configuration[i,j] == 2:
                        lmleft, lmright, lmabove, lmbelow = label_matrix[i,(j-1)%L], label_matrix[i,(j+1)%L], label_matrix[(i-1)%L,j], label_matrix[(i+1)%L,j]
                        lmarr = [lmleft, lmright, lmabove, lmbelow]
                        value = num_of_zero_in_list(lmarr)
                        numbers = []
                        sites = []

                        if value == 4: #Zero neighbor
                            label += 1
                            label_dict[label] = [(i,j)] #make new key and contain it's coordinate there.

                        elif value == 3: #One neighbor
                            for k in lmarr:
                                if k != 0: #one neighbor is occupied
                                    label_dict[k] += [(i,j)] #append a coordinate

                        elif value == 2: #Two neighbors
                            sites = near_by_sites(lmarr)
                            if sites[0] != sites[1]: #labels are different
                                label_dict[sites[0]] += [(i,j)] + label_dict[sites[1]]
                                label_dict.pop(sites[1])
                                #for k in sites: #with respect to occupied sites,
                                #    numbers.append(num_in_dict(label_dict, k))
                                #if numbers[0] > numbers[1]:
                                #    label_dict[sites[0]] += [(i,j)] + label_dict[sites[1]]
                                #    label_dict.pop(sites[1])
                                #else:
                                #    label_dict[sites[1]] += [(i,j)] + label_dict[sites[0]] 
                                #    label_dict.pop(sites[0])
                            elif sites[0] == sites[1]: #label of two sites are equal.
                                label_dict[sites[0]] += [(i,j)]

                        elif value == 1: #if three neighbors are preoccupied.
                            sites = near_by_sites(lmarr)
                            multiplicity = multi(sites)
                            if 3 in multiplicity: #multiplicity = [3,3,3]
                                label_dict[sites[0]] += [(i,j)]
                            elif 1 and 2 in multiplicity: #multiplicity = [1,2,2]
                                N = list(set(sites))
                                label_dict[N[0]] += [(i,j)] + label_dict[N[1]]
                                label_dict.pop(N[1])
                                #if num_in_dict(label_dict, N[0]) > num_in_dict(label_dict, N[1]):
                                #    label_dict[N[0]] += [(i,j)] + label_dict[N[1]]
                                #    label_dict.pop(N[1])
                                #else:
                                #    label_dict[N[1]] += [(i,j)] + label_dict[N[0]]
                                #    label_dict.pop(N[0])                            
                            elif 1 in multiplicity and 2 not in multiplicity: #3개 모두 다른 label multiplicity = [1,1,1]
                                for q in range(len(sites)):
                                    numbers.append((num_in_dict(label_dict, sites[q]), q))
                                numbers.sort() 
                                label_dict[sites[numbers[2][1]]] += [(i,j)] + label_dict[sites[numbers[0][1]]] + label_dict[sites[numbers[1][1]]]
                                label_dict.pop(sites[numbers[0][1]])
                                label_dict.pop(sites[numbers[1][1]])

                        elif value == 0: #all neighbor sites is occupied.
                            multiplicity = multi(lmarr)
                            N = list(set(lmarr))
                            if 4 in multiplicity: #multiplicity = [4,4,4,4]
                                label_dict[lmleft] += [(i,j)]
                            elif 1 and 3 in multiplicity: #multiplicity = [1,3,3,3]
                                label_dict[N[0]] += [(i,j)] + label_dict[N[1]]
                                label_dict.pop(N[1])
                                #if num_in_dict(label_dict, N[0]) > num_in_dict(label_dict, N[1]):
                                #    label_dict[N[0]] += [(i,j)] + label_dict[N[1]]
                                #    label_dict.pop(N[1])
                                #else:
                                #    label_dict[N[1]] += [(i,j)] + label_dict[N[0]]
                                #    label_dict.pop(N[0])   
                            elif 2 in multiplicity and 1 not in multiplicity: #multiplicity = [2,2, 2,2]
                                label_dict[N[0]] += [(i,j)] + label_dict[N[1]]
                                label_dict.pop(N[1])

                                #if num_in_dict(label_dict, N[0]) > num_in_dict(label_dict, N[1]):
                                #    label_dict[N[0]] += [(i,j)] + label_dict[N[1]]
                                #    label_dict.pop(N[1])
                                #else:
                                #    label_dict[N[1]] += [(i,j)] + label_dict[N[0]]
                                #    label_dict.pop(N[0])
                            elif 1 and 2 in multiplicity and 3 not in multiplicity: #multiplicity = [1,1,2,2]
                                for l in range(len(N)):
                                    numbers.append((num_in_dict(label_dict, N[l]), l))
                                numbers.sort()
                                label_dict[N[numbers[2][1]]] += [(i,j)] + label_dict[N[numbers[0][1]]] + label_dict[N[numbers[1][1]]]
                                label_dict.pop(N[numbers[0][1]])
                                label_dict.pop(N[numbers[1][1]])     
                            elif 1 in multiplicity and 2 not in multiplicity and 3 not in multiplicity: #multiplicity = [1,1,1,1]
                                for m in range(len(lmarr)):
                                    numbers.append((num_in_dict(label_dict, lmarr[m]), m))
                                numbers.sort()
                                label_dict[lmarr[numbers[3][1]]] += [(i,j)] + label_dict[lmarr[numbers[0][1]]] + label_dict[lmarr[numbers[1][1]]] + label_dict[lmarr[numbers[2][1]]]
                                label_dict.pop(lmarr[numbers[0][1]])
                                label_dict.pop(lmarr[numbers[1][1]])    
                                label_dict.pop(lmarr[numbers[2][1]])    

                        configuration[i,j] = 1
                        label_matrix = dict_to_matrix(L, label_dict)
                        gc_ratio.append(giant_cluster_size(label_dict)/(L**2))
                        mean_cluster.append(cluster_sizes(label_dict)/(L**2))
                        break
                else:
                    continue
                break 
    return configuration, label_matrix, gc_ratio, mean_cluster

In [4]:
def ensemble_union_find(lattice_size, number_of_ensemble):
    '''
    By computing percolation with number_of_ensemble times, return ensemble averaged values.
    '''
    alls = []
    sus_alls = []
    for n in range(number_of_ensemble):
        m, lab, gc, mc = union_find(lattice_size, lattice_size**2)
        alls.append(np.array(gc))
        sus_alls.append(np.array(mc))
    # Z는 average magnetization
    Z = np.zeros(len(gc))
    Z2 = np.zeros(len(gc))
    Z4 = np.zeros(len(gc))
    SS = np.zeros(len(mc))
    
    for i in range(number_of_ensemble):
        Z += alls[i]
        Z2 += (alls[i])**2
        Z4 += (alls[i])**4
        SS += sus_alls[i]
        
    average = Z / number_of_ensemble
    suscept = SS / number_of_ensemble
    cumulant = np.ones(len(gc)) - (Z4/number_of_ensemble) / (3*(Z2/number_of_ensemble)**2) 
    
    return average, suscept, cumulant

## Finite size scaling
Set $x$ axis as $(p-p_c)L^{1/\nu}$ and  $y$ axis as $S_{\infty}L^{\beta/\nu}$ for fraction of giant cluster. ($S_{\infty}$ is fraction of giant cluster at infinite lattice.)<br>
Set $x$ axis as $(p-p_c)L^{1/\nu}$ and $y$ axis as $\chi L^{\gamma/\nu}$ for its susceptibility. <br>
$\beta = 5/36$, $\nu = 4/3$, and $\gamma = 43/18$.

In [9]:
pc = 0.595 #percolation threshold.

In [10]:
def percolation_finte_size_scaling(linear_lattice_size, average_fraction, susceptibility, percolation_threshold):
    x = np.arange(0,linear_lattice_size**2)/(linear_lattice_size**2)
    x_pc = x - percolation_threshold
    X = x_pc * linear_lattice_size**(3/4)
    
    scaled_Af = average_fraction * linear_lattice_size**(5/36 * 3/4)
    scaled_Sus = susceptibility * linear_lattice_size**(-43/18 * 3/4)
    
    return X, scaled_Af, scaled_Sus